<a href="https://colab.research.google.com/github/AaronGoldsmith/gpt-util/blob/main/gpt_weather_function_calling_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%pip install openai pyairtable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from getpass import getpass
import openai
import json

openai.api_key = getpass("API KEY")


API KEY··········


In [4]:
import ossaudiodev
import requests
from requests.structures import CaseInsensitiveDict
geoapify_key = getpass("GeoAPIfy API key: ")

GeoAPIfy API key: ··········


In [5]:
owm_api_key = getpass("Open Weather Map API key: ")

Open Weather Map API key: ··········


In [6]:
import logging
import json
import openai

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
# logger.setLevel(logging.DEBUG)

In [7]:
def request_forecast_at_location(lat, lon):
  query = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={owm_api_key}"

  try:
    response = requests.get(query)
  except Exception as E:
    logging.warn(E)
  return json.dumps(response.json()["main"])

In [9]:
from pyairtable import Table
import datetime

In [25]:
airtable_api_key = getpass("Airtable API key: ")

Airtable API key: ··········


In [55]:
APPLICATION_ID = "appjyAU0r4Lw6Cj6K"
TABLE_ID = "tblPxw9vYfQUt7g19"

table = Table(airtable_api_key, APPLICATION_ID, TABLE_ID)

def add_weather_to_airtable(location, temperature, weather_summary):
  current_date = str(datetime.date.today())
  payload = {
    "Location": location,
    "Date": current_date,
    "Temperature": temperature,
    "Forecast": weather_summary,
  }
  table.create(payload)
  return f"created with payload: {payload}"


In [56]:
add_weather_to_airtable("Oakland, CA","61","Partly cloudy")

"created with payload: {'Location': 'Oakland, CA', 'Date': '2023-06-20', 'Temperature': '61', 'Forecast': 'Partly cloudy'}"

In [20]:
def reverse_geo_lookup(location, limit=5):
  url = f"https://api.geoapify.com/v1/geocode/search?text={location}&apiKey={geoapify_key}"

  headers = CaseInsensitiveDict()
  headers["Accept"] = "application/json"

  try:
    response = requests.get(url, headers=headers)
    features = response.json()['features']
    results = []

    for result in features[:limit]:
      prop = result["properties"]
      if 'city' in prop:
        results.append({
          "city": prop["city"],
          "state": prop["state"],
          "formatted": prop["formatted"],
          "lat": prop["lat"],
          "lon": prop["lon"],
          "popularity": prop["rank"]["popularity"]
        })
  except Exception as e:
    print(e)

  return json.dumps(results)

In [34]:
function_descriptions = [
        {
            "name": "request_forecast_at_location",
            "description": "Get the current weather at a given latitude / longitutde.",
            "parameters": {
                "type": "object",
                "properties": {
                    "lat": {
                        "type": "string",
                        "description": "The latitudinal coordinate of the location",
                    },
                    "lon": {
                        "type": "string",
                        "description": "The longitudinal coordinate of the location",
                    },
                },
                "required": ["lat", "lon"],
            },
        },
         {
          "name": "reverse_geo_lookup",
          "description":
          "Lookup a latitude and longitude from a city, address, or zipcode",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "the location to perform the reverse geo_lookup on"
              },
              "limit":{
                  "type": "string",
                  "description": "the number of results to return (default 1)"
              },
            },
            "required": ["location"],
          }
        },
        {
         "name": "add_weather_to_airtable",
          "description": "Useful when you need to save weather details to  to Airtable",
          "parameters": {
            "type": "object",
            "properties": {
              "location": {
                "type": "string",
                "description": "The city/state  for the weather report."
              },
              "temperature": {
                "type": "string",
                "description": "the temperature in degrees farenheit"
              },
              "weather_summary": {
                "type": "string",
                "description": "Current weather summary at a specified location"
              },
            },
            "required": ["location", "temperature", "weather_summary"],
          },
        }
    ]

In [36]:
def handle_function_call(response_message):
  available_functions = {
            "request_forecast_at_location": request_forecast_at_location,
            "reverse_geo_lookup": reverse_geo_lookup,
            "add_weather_to_airtable": add_weather_to_airtable
  }
  function_name = response_message["function_call"]["name"]
  if function_name in available_functions.keys():
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])


  if function_name == "reverse_geo_lookup":
    if isinstance(function_args, list):
      function_args = function_args[0]
    function_response = function_to_call(
        location=function_args.get("location")
    )
  elif function_name =="request_forecast_at_location":
    function_response = function_to_call(
        lat=function_args.get("lat"),
        lon=function_args.get("lon")
    )
  elif function_name =="add_weather_to_airtable":
    location=function_args.get("location")
    temperature=function_args.get("temperature")
    weather_summary=function_args.get("weather_summary")
    print(location, temperature, weather_summary)
    function_response = function_to_call(
        location=function_args.get("location"),
        temperature=function_args.get("temperature"),
        weather_summary=function_args.get("weather_summary"),
    )
  else:
    logging.info("function call not in list\n", function_name)
    logging.info(function_args)
    function_response = "Unknown"
  return function_response


In [13]:
MODEL_NAME = "gpt-3.5-turbo-0613"

In [14]:
def create_chat_response(messages, include_functions=False):
    if include_functions:
        response = openai.ChatCompletion.create(
            model=MODEL_NAME,
            messages=messages,
            functions=function_descriptions,
            function_call="auto"
        )
    else:
        response = openai.ChatCompletion.create(
            model=MODEL_NAME,
            messages=messages
        )
    return response


In [67]:
def run_conversation(query):
    messages = [{"role": "user", "content": query}]
    response = create_chat_response(messages, include_functions=True)

    logging.info(query)
    response_message = response["choices"][0]["message"]

    while response_message.get("function_call"):
        # call the function
        function_name = response_message["function_call"]["name"]
        logging.info(f"...Making function call to {function_name}")

        # add the response to the list of messages
        function_response = handle_function_call(response_message)
        logging.info(f"\trecieved response from {function_name}" )
        # logging.info(function_response.replace('\n',''))

        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        # Call Chat endpoint with the messages (now containing function response)
        second_response = create_chat_response(messages, include_functions=True)
        response_message = second_response["choices"][0]["message"]
        messages.append(response_message)

        if response_message.get("function_call"):
          function_name = response_message["function_call"]["name"]
          logging.info(f"\nNext function to call: {function_name}")
        else: return second_response


    logging.info(messages[len(messages)-1])
    final_response = create_chat_response(messages)
    final_tokens = final_response["usage"]["total_tokens"]

    logging.info("Done")
    logging.info("Total tokens used: %s", final_tokens )
    return final_response


In [ ]:
request_forecast_at_location("37.8044557","-122.271356")

'{"coord": {"lon": -122.2714, "lat": 37.8045}, "weather": [{"id": 801, "main": "Clouds", "description": "few clouds", "icon": "02d"}], "base": "stations", "main": {"temp": 292.09, "feels_like": 291.58, "temp_min": 288.1, "temp_max": 297.24, "pressure": 1016, "humidity": 59}, "visibility": 10000, "wind": {"speed": 12.35, "deg": 290, "gust": 16.98}, "clouds": {"all": 20}, "dt": 1687216692, "sys": {"type": 2, "id": 2042798, "country": "US", "sunrise": 1687178818, "sunset": 1687232032}, "timezone": -25200, "id": 5378538, "name": "Oakland", "cod": 200}'

In [70]:
result = run_conversation("Can you save the temperature in San Jose, CA? I'd like it in Farenheit")

INFO:root:Can you save the temperature in San Jose, CA? I'd like it in Farenheit
INFO:root:...Making function call to request_forecast_at_location
INFO:root:	recieved response from request_forecast_at_location
INFO:root:
Next function to call: add_weather_to_airtable
INFO:root:...Making function call to add_weather_to_airtable


San Jose, CA 68 Partly cloudy


INFO:root:	recieved response from add_weather_to_airtable


In [71]:
result["choices"][0]["message"]["content"]

'The temperature in San Jose, CA is 68 degrees Fahrenheit and the weather is partly cloudy. I have saved this information in the Airtable.'